In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 3, Finished, Available, Finished, False)

In [2]:
df = spark.read.format("csv").option("header","true").load("Files/Bronzebronze_collections/collections.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Bronzebronze_collections/collections.csv".
display(df)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 4, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, a44aad0b-494a-475c-ad20-6442a9173f08)

In [3]:
df_sales = spark.read.parquet("Files/Silver Layer Files/silver_Sales") \
    .select("SalesID", "InvoiceAmount", "InvoiceDate", "DueDate")
display(df_sales)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 5, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 91e4c56b-f344-403d-9a76-44babc812780)

In [4]:
from pyspark.sql.functions import col, trim, upper
from pyspark.sql.types import DoubleType

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 6, Finished, Available, Finished, False)

In [5]:
df_silver = (
    df
    .withColumn("CollectionID", trim(upper(col("CollectionID"))))
    .withColumn("CustomerID", trim(upper(col("CustomerID"))))
    .withColumn("InvoiceID", trim(upper(col("InvoiceID"))))
    .withColumn("AmountPaid", col("AmountPaid").cast(DoubleType()))
)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 7, Finished, Available, Finished, False)

In [6]:
df_silver = df_silver.filter(col("AmountPaid") > 0)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 8, Finished, Available, Finished, False)

In [7]:
from pyspark.sql.functions import to_date

df_silver = df_silver.withColumn(
    "CollectionDate",
    to_date(col("CollectionDate"), "dd-MM-yyyy")
)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 9, Finished, Available, Finished, False)

In [8]:
display(df_silver)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 10, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 0d41c2ef-e56b-4f96-801c-3441cef5ff31)

In [9]:
df_joined = df_silver.join(
    df_sales,
    df_silver["InvoiceID"] == df_sales["SalesID"],
    "left"
)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 11, Finished, Available, Finished, False)

In [10]:
from pyspark.sql.functions import datediff

df_joined = df_joined.withColumn(
    "DaysToCollect",
    datediff(col("CollectionDate"), col("InvoiceDate"))
)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 12, Finished, Available, Finished, False)

In [11]:
display(df_joined)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 13, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 389e6e12-9532-43a3-bf54-6afd72a795d4)

In [12]:
from pyspark.sql.functions import when

df_joined = df_joined.withColumn(
    "IsDelayed",
    when(col("CollectionDate") > col("DueDate"), 1).otherwise(0)
)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 14, Finished, Available, Finished, False)

In [13]:
df_joined = df_joined.withColumn(
    "InvoiceStatus",
    when(col("AmountPaid") >= col("InvoiceAmount"), "Paid")
    .when(col("AmountPaid") < col("InvoiceAmount"), "Partial")
    .otherwise("Open")
)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 15, Finished, Available, Finished, False)

In [14]:
display(df_joined)

StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 16, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 905339e2-9da8-49a7-8c6a-ef0b6e3709ec)

In [15]:
silver_path = "abfss://Fabric_Test@onelake.dfs.fabric.microsoft.com/Test_Lakehouse.Lakehouse/Files/Silver Layer Files"
df_joined.write.mode("overwrite") \
    .parquet(f"{silver_path}/silver_collection")


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 17, Finished, Available, Finished, False)

In [16]:
df_check = spark.read.parquet(f"{silver_path}/silver_collection")
display(df_check)


StatementMeta(, 543e394c-e41f-4416-8850-705f2bbc719d, 18, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, db76e1f4-3244-4b7d-b34a-0d457fac83e9)